In [1]:
import oracledb
import pandas as pd

`ssh -l <username> -L 127.0.0.1:1522:dbhost.students.cs.ubc.ca:1522 remote.students.cs.ubc.ca`

https://www.students.cs.ubc.ca/~cs-368/resources/connecting-to-db.html

In [ ]:
username = ...
cwl = ...

dsn = oracledb.makedsn("localhost", 1522, service_name="stu")
connection = oracledb.connect(user=f"ora_{username}", password=f"a{cwl}", dsn=dsn)

cur = connection.cursor()
# SQL code
for row in cur.execute("select * from titles"):
    print(row)
cur.close()

In [3]:
dsn = oracledb.makedsn("localhost", 1522, service_name="stu")
connection = oracledb.connect(user="ora_nick2003", password="a45695970", dsn=dsn)
cur = connection.cursor()

# Execute SQL query
cur.execute(
"""
SELECT t1.TYPE, SUM(t1.PRICE) 
FROM titles t1
GROUP BY t1.TYPE
"""
)

# Fetch all results
rows = cur.fetchall()

# Get column names from cursor
columns = [desc[0] for desc in cur.description]

# Create DataFrame
df = pd.DataFrame(rows, columns=columns)

cur.execute(
"""
SELECT t1.TYPE, SUM(t1.ADVANCE) 
FROM titles t1
GROUP BY t1.TYPE
"""
)

rows2 = cur.fetchall()
columns2 = [desc[0] for desc in cur.description]
df2 = pd.DataFrame(rows2, columns=columns2)

# Close cursor and connection
cur.close()
connection.close()

# Display DataFrame
display(df)
display(df2)

,TYPE,SUM(T1.PRICE)
0,psychology,68.0
1,trad_cook,48.0
2,popular_comp,43.0
3,None,NaN
4,business,55.0
5,mod_cook,23.0


,TYPE,SUM(T1.ADVANCE)
0,psychology,21275.0
1,trad_cook,19000.0
2,popular_comp,15000.0
3,None,NaN
4,business,25125.0
5,mod_cook,15000.0


In [4]:
dsn = oracledb.makedsn("localhost", 1522, service_name="stu")
connection = oracledb.connect(user="ora_nick2003", password="a45695970", dsn=dsn)
cur = connection.cursor()

# Execute SQL query
cur.execute(
"""
SELECT t1.TYPE, SUM(t1.ADVANCE) 
FROM titles t1
GROUP BY t1.TYPE
"""
)

# Fetch all results
rows = cur.fetchall()

# Get column names from cursor
columns = [desc[0] for desc in cur.description]

# Create DataFrame
df = pd.DataFrame(rows, columns=columns)

# Close cursor and connection
cur.close()
connection.close()

# Display DataFrame
display(df)

,TYPE,SUM(T1.ADVANCE)
0,psychology,21275.0
1,trad_cook,19000.0
2,popular_comp,15000.0
3,None,NaN
4,business,25125.0
5,mod_cook,15000.0
